In [2]:
!pip install keras
!pip install gensim -U
!pip install nltk
!pip install -U sentence-transformers
!pip install git+https://github.com/facebookresearch/SentEval

The system cannot find the path specified.


  Using cached https://files.pythonhosted.org/packages/15/89/b785d557e3c806abc8beaae664571d71e8c4eb736a2c32b69aba9932cbd1/gensim-4.0.1-cp37-cp37m-win_amd64.whl
  Found existing installation: gensim 3.8.1
    Uninstalling gensim-3.8.1:
      Successfully uninstalled gensim-3.8.1


The system cannot find the path specified.


The system cannot find the path specified.


Requirement already up-to-date: sentence-transformers in c:\users\user\anaconda3\lib\site-packages (2.0.0)


The system cannot find the path specified.


  Cloning https://github.com/facebookresearch/SentEval to c:\users\user\appdata\local\temp\pip-req-build-y_wtji8k
  Created wheel for SentEval: filename=SentEval-0.1.0-cp37-none-any.whl size=35245 sha256=a4d09b466c9e8eb02a5cb607bad74d252b78adb13fac86217007fb239705ca63
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-df59ewj3\wheels\12\d1\1e\7e46b4b6d1b480028b6a9dec849bf801846023d4c93d172bbc
Successfully built SentEval


The system cannot find the path specified.
  Running command git clone -q https://github.com/facebookresearch/SentEval 'C:\Users\user\AppData\Local\Temp\pip-req-build-y_wtji8k'


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import gensim
import warnings
import logging
warnings.filterwarnings("ignore")
import sys
import io
from gensim.models import KeyedVectors as evalVec
FIN_PATH = 'fin-how2-ted.bin'

In [12]:
import senteval
ft_model = gensim.models.Word2Vec.load("ft2.model")

In [13]:
filepath = "sentences.txt"
dataset=[]
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        dataset.append(line)
dataset=dataset[:5000]      
len(dataset[:5000])

5000

In [14]:
# SentEval prepare and batcher
def prepare(params, samples):
    _, params.word2id = create_dictionary(samples)
    params.word_vec = get_wordvec(PATH_TO_VEC, params.word2id)
    params.wvec_dim = 100
    return

In [15]:
PATH_TO_DATA = "sent-eval-data"
embed_size = 100
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

In [16]:
params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}

In [17]:
def l2_norm(x):
    return np.sqrt(np.sum(x**2))

def div_norm(x):
    norm_value = l2_norm(x)
    if norm_value > 0:
        return x * ( 1.0 / norm_value)
    else:
        return x

def getSentenceEmbedding(sent, model):
    sentArr=sent.split(" ")
    start = np.zeros(embed_size)
    for word in sentArr:
        word_embed=np.array([model.wv[word]])
        start+=div_norm(word_embed)
    return start/len(sentArr)

# def batcher(params, batch):
#     embeddings = []
#     for sent in dataset:
#         sentvec = getSentenceEmbedding(sent,ft_model)
#         embeddings.append(sentvec)
#     embeddings = np.vstack(embeddings)
#     return embeddings

In [18]:
# Create dictionary
def create_dictionary(sentences, threshold=0):
    words = {}
    for s in sentences:
        for word in s:
            words[word] = words.get(word, 0) + 1

    if threshold > 0:
        newwords = {}
        for word in words:
            if words[word] >= threshold:
                newwords[word] = words[word]
        words = newwords
    words['<s>'] = 1e9 + 4
    words['</s>'] = 1e9 + 3
    words['<p>'] = 1e9 + 2

    sorted_words = sorted(words.items(), key=lambda x: -x[1])  # inverse sort
    id2word = []
    word2id = {}
    for i, (w, _) in enumerate(sorted_words):
        id2word.append(w)
        word2id[w] = i

    return id2word, word2id

# Get word vectors from vocabulary (glove, word2vec, fasttext ..)
def get_wordvec(path_to_vec, word2id):
    word_vec = {}
    for wd in word2id:
        word_vec[wd] = ft_model.wv[wd].T
    logging.info('Found {0} words with word vectors, out of {1} words'.format(len(word_vec), len(word2id)))
    return word_vec

In [15]:
PATH_TO_VEC = 'ft2.model'
#se = senteval.engine.SE(params, batcher, prepare)
se = evalVec.load_word2vec_format(FIN_PATH, binary=True)

def batcher(params, batch):
    batch = []
    for x in dataset:
        if(x!="" and x!=" "):
            batch.append(x.split(" "))
    embeddings = []
    for sent in batch:
        sentvec = []
        for word in sent:
            if word in params.word_vec:
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            sentvec.append(vec)
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)

    embeddings = np.vstack(embeddings)
    return embeddings

In [16]:
se.evaluate_word_pairs('eval/wordsim353.tsv',dummy4unknown=True)

((0.6238773466616108, 1.7963237724170263e-39),
 SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45),
 0.0)

In [17]:
se.evaluate_word_pairs('eval/simlex-999.tsv',dummy4unknown=True)

((0.448323335546951, 1.480562485841737e-50),
 SpearmanrResult(correlation=0.4371714088057533, pvalue=6.926977701395097e-48),
 0.10010010010010009)